Copyright (c) 2020. Cognitive Scale Inc. All rights reserved.
Licensed under CognitiveScale Example Code [License](https://github.com/CognitiveScale/cortex-certifai-examples/blob/7998b8a481fccd467463deb1fc46d19622079b0e/LICENSE.md)

# Introduction

This is the third notebook in this example of how to explain models using Certifai. If you have not already done so, please run the [first notebook](patient-readmission-train.ipynb) to train the models to be explained and the [second notebook](patient-readmission-explain-scan.ipynb) to scan the models.

In this notebook, we will:
1. Load the previously saved explanations reports
2. Convert the counterfactuals into a dataframe and display them


In [1]:
import numpy as np
import pandas as pd
from pprint import pprint

from certifai.scanner.report_reader import ScanReportReader
from certifai.scanner.explanation_utils import explanations, construct_explanations_dataframe
from certifai.scanner.builder import ExplanationType
from IPython.display import display, Markdown

# Loading the Explanations Report

To load the report, we need to know the use case ID ('readmission') and the scan ID.

List the available use cases, and the scans within the 'readmission' use case.

In [2]:
reader = ScanReportReader("reports")
reader.list_usecases()
scans = reader.list_scans('readmission')
data=[[s['date'], ', '.join(s['reportTypes']), s['id']] for s in scans]
df = pd.DataFrame(data, columns=['date', 'evals', 'scan id']).sort_values(by=['date'], ascending=False)
print(df)

              date                                      evals       scan id
0  20201016T132038                                explanation  5683ba38b998
1  20201011T151543  atx, fairness, robustness, explainability  4b13c55a4131
2  20201011T141903                                explanation  7a2e8b64cbd6


Locate the latest explanation scan and load it.

In [3]:
latest_explanation = df[df.evals == 'explanation'].iloc[0]
result = reader.load_scan('readmission', latest_explanation['scan id'])

# Extract the explanations

In this section we'll construct a dataframe containing all of the original instances, and their counterfactuals. We'll then print out the first two for the logit and mlp models.

Construct the dataframe containing the original instances and the counterfactual instances.

In [4]:
all_explanations = explanations(result)
df_all = construct_explanations_dataframe(all_explanations)
display(df_all[df_all['row']<3])

,model,row,instance,cf_num,cf_type,prediction,fitness,age,time_in_hospital,num_lab_procedures,...,troglitazone,tolazamide,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed
0,logit,1,original,0,original prediction,1,0.000000,85,3,47,...,No,No,Up,No,No,No,No,No,Ch,Yes
1,logit,1,counterfactual,1,prediction changed,0,1.262681,85,3,47,...,No,No,Up,No,No,No,No,No,Ch,Yes
2,logit,2,original,0,original prediction,0,0.000000,55,3,9,...,No,No,Steady,No,No,No,No,No,No,Yes
3,logit,2,counterfactual,1,prediction changed,1,0.479862,55,4,9,...,No,No,Steady,No,No,No,No,No,No,Yes
200,mlp,1,original,0,original prediction,1,0.000000,85,3,47,...,No,No,Up,No,No,No,No,No,Ch,Yes
201,mlp,1,counterfactual,1,prediction changed,0,1.000000,85,3,47,...,No,No,Up,Up,No,No,No,No,Ch,Yes
202,mlp,2,original,0,original prediction,0,0.000000,55,3,9,...,No,No,Steady,No,No,No,No,No,No,Yes
203,mlp,2,counterfactual,1,prediction changed,1,1.000000,55,3,9,...,No,No,Steady,No,No,No,No,No,No,Yes


Print out differences for the first explained prediction for the logit and mlp models.

In [5]:
max_displayed = 2
pd.set_option('display.max_columns', None) # print all cols

for model in ['logit', 'mlp']:
    for row in range(1,max_displayed+1):
        df_instances = df_all[(df_all['model']==model) & (df_all['row']==row)]
        df_original = df_instances[df_instances['instance']=='original'].reset_index(drop=True)
        display(Markdown(f'### Explanation of model {model} prediction row {row}\n'))
        display(Markdown('**Original Instance**'))
        display(df_original)
        orig_prediction=df_original['prediction'].iloc[0]
        display(Markdown(f'**Original Prediction**: {"Readmitted" if orig_prediction == 1 else "Not Readmitted"}'))
        display(Markdown('### Counterfactual Changes'))
        df_cfs = df_instances[df_instances['instance']=='counterfactual'].reset_index(drop=True)
        cf_changes = df_cfs.where(df_cfs != df_original).dropna(axis=1)
        display(pd.concat([df_original[cf_changes.columns], cf_changes]))


### Explanation of model logit prediction row 1


**Original Instance**

,model,row,instance,cf_num,cf_type,prediction,fitness,age,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,race,gender,diag_1,diag_2,diag_3,max_glu_serum,A1Cresult,metformin,repaglinide,nateglinide,chlorpropamide,glimepiride,acetohexamide,glipizide,glyburide,tolbutamide,pioglitazone,rosiglitazone,acarbose,miglitol,troglitazone,tolazamide,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed
0,logit,1,original,0,original prediction,1,0.0,85,3,47,0,13,0,0,1,9,Caucasian,Female,Genitourinary,Other,Diabetes,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Up,No,No,No,No,No,Ch,Yes


**Original Prediction**: Readmitted

### Counterfactual Changes

...

**Original Instance**

,model,row,instance,cf_num,cf_type,prediction,fitness,age,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,race,gender,diag_1,diag_2,diag_3,max_glu_serum,A1Cresult,metformin,repaglinide,nateglinide,chlorpropamide,glimepiride,acetohexamide,glipizide,glyburide,tolbutamide,pioglitazone,rosiglitazone,acarbose,miglitol,troglitazone,tolazamide,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed
0,mlp,2,original,0,original prediction,0,0.0,55,3,9,1,22,1,0,0,4,Caucasian,Male,Other,Circulatory,Diabetes,Norm,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,No,Yes


**Original Prediction**: Not Readmitted

### Counterfactual Changes

,instance,cf_num,cf_type,prediction,fitness,diag_1
0,original,0,original prediction,0,0.0,Other
0,counterfactual,1,prediction changed,1,1.0,Diabetes
